<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Selection using Wrapper Methods</h4>
    <p style="font-size: 20px;">NBA API Data (2022-2024)</p>
</div>

<a name="Feature-Selection"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Wrapper Methods for Total Points](#1.-Wrapper-Methods-for-Total-Points)**

- [1.1. Linear Regression](#1.1.-Linear-Regression)

- [1.2. Random Forest](#1.2.-Random-Forest)

**[2. Wrapper Methods for Plus Minus](#2.-Wrapper-Methods-for-Plus-Minus)**

- [2.1. Linear Regression](#2.1.-Linear-Regression)

- [2.2. Random Forest](#2.2.-Random-Forest)

**[3. Wrapper Methods for Game Winner](#3.-Wrapper-Methods-for-Game-Winner)**

- [3.1. Linear Regression](#3.1.-Linear-Regression)

- [3.2. Random Forest](#3.2.-Random-Forest)

# Setup

[Return to top](#Feature-Selection)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Data

[Return to top](#Feature-Selection)

In [2]:
# load all the rolling data
boxscores = pd.read_csv('../../data/processed/nba_team_matchups_rolling_box_scores_2022_2024_r05.csv')
opponent = pd.read_csv('../../data/processed/nba_team_matchups_rolling_opponent_impact_box_scores_2022_2024_r05.csv')
advanced = pd.read_csv('../../data/processed/nba_team_matchups_rolling_adv_stats_2021_2024_r05.csv')
hustle = pd.read_csv('../../data/processed/nba_team_matchups_rolling_hustle_stats_2021_2024_r05.csv')
miscbox = pd.read_csv('../../data/processed/nba_team_matchups_rolling_misc_box_stats_2021_2024_r05.csv')
factor = pd.read_csv('../../data/processed/nba_team_matchups_rolling_four_factor_stats_2021_2024_r05.csv')
scoring = pd.read_csv('../../data/processed/nba_team_matchups_rolling_scoring_box_stats_2021_2024_r05.csv')
track = pd.read_csv('../../data/processed/nba_team_matchups_rolling_player_track_box_stats_2021_2024_r05.csv')

In [3]:
# merge all the features
cols_to_drop = ['GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS', 'HOME_TEAM_NAME',
                'SEASON_ID', 'GAME_DATE', 'AWAY_TEAM_NAME', 'ROLL_HOME_PTS', 'ROLL_AWAY_PTS']

features_df = boxscores.merge(opponent.drop([col for col in cols_to_drop if col in opponent.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(advanced.drop([col for col in cols_to_drop if col in advanced.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(hustle.drop([col for col in cols_to_drop if col in hustle.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(miscbox.drop([col for col in cols_to_drop if col in miscbox.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(factor.drop([col for col in cols_to_drop if col in factor.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(scoring.drop([col for col in cols_to_drop if col in scoring.columns], axis=1), how='left', on='GAME_ID')
features_df = features_df.merge(track.drop([col for col in cols_to_drop if col in track.columns], axis=1), how='left', on='GAME_ID')

print(features_df.shape)
features_df.head()

(3172, 264)


,GAME_ID,GAME_RESULT,TOTAL_PTS,PLUS_MINUS,HOME_TEAM_NAME,SEASON_ID,GAME_DATE,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT_x,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST_x,ROLL_HOME_STL,ROLL_HOME_BLK_x,ROLL_HOME_TOV,ROLL_HOME_PF_x,...,ROLL_HOME_UFGA,ROLL_HOME_UFG_PCT,ROLL_HOME_FG_PCT_y,ROLL_HOME_DFGM,ROLL_HOME_DFGA,ROLL_HOME_DFG_PCT,ROLL_AWAY_DIST,ROLL_AWAY_ORBC,ROLL_AWAY_DRBC,ROLL_AWAY_RBC,ROLL_AWAY_TCHS,ROLL_AWAY_SAST,ROLL_AWAY_FTAST,ROLL_AWAY_PASS,ROLL_AWAY_AST_y,ROLL_AWAY_CFGM,ROLL_AWAY_CFGA,ROLL_AWAY_CFG_PCT,ROLL_AWAY_UFGM,ROLL_AWAY_UFGA,ROLL_AWAY_UFG_PCT,ROLL_AWAY_FG_PCT_y,ROLL_AWAY_DFGM,ROLL_AWAY_DFGA,ROLL_AWAY_DFG_PCT
0,22100002,0,235,-7.0,Los Angeles Lakers,2021-22,2021-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22100001,1,231,23.0,Milwaukee Bucks,2021-22,2021-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22100007,1,253,11.0,Memphis Grizzlies,2021-22,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22100008,1,230,18.0,Minnesota Timberwolves,2021-22,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22100009,0,214,-20.0,New Orleans Pelicans,2021-22,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# load, filter (by time) and scale data
pts_features, pm_features, res_features, test_set_obs = utl.load_and_scale_data(
    input_data=features_df,
    seasons_to_keep=['2021-22', '2022-23', '2023-24'],
    training_season='2021-22',
    feature_prefixes=['ROLL_', 'ROLLDIFF_'],
    scaler_type='minmax', 
    scale_target=False
)

Season 2021-22: 1186 games
Season 2022-23: 1181 games
Season 2023-24: 691 games
Total number of games across sampled seasons: 3058 games


In [5]:
pts_features.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT_x,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST_x,ROLL_HOME_STL,ROLL_HOME_BLK_x,ROLL_HOME_TOV,ROLL_HOME_PF_x,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT_x,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,...,ROLL_HOME_UFG_PCT,ROLL_HOME_FG_PCT_y,ROLL_HOME_DFGM,ROLL_HOME_DFGA,ROLL_HOME_DFG_PCT,ROLL_AWAY_DIST,ROLL_AWAY_ORBC,ROLL_AWAY_DRBC,ROLL_AWAY_RBC,ROLL_AWAY_TCHS,ROLL_AWAY_SAST,ROLL_AWAY_FTAST,ROLL_AWAY_PASS,ROLL_AWAY_AST_y,ROLL_AWAY_CFGM,ROLL_AWAY_CFGA,ROLL_AWAY_CFG_PCT,ROLL_AWAY_UFGM,ROLL_AWAY_UFGA,ROLL_AWAY_UFG_PCT,ROLL_AWAY_FG_PCT_y,ROLL_AWAY_DFGM,ROLL_AWAY_DFGA,ROLL_AWAY_DFG_PCT,TOTAL_PTS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,0.745,0.522,0.296,0.753,0.758,0.580,0.731,0.805,0.878,0.535,0.571,0.292,0.478,0.612,1.000,1.000,0.600,0.661,0.577,0.586,0.202,0.704,0.526,0.176,1.000,...,0.761,0.753,0.909,1.000,0.352,0.665,0.613,0.119,0.252,0.530,0.250,0.714,0.531,0.500,0.595,0.704,0.564,0.409,0.190,0.738,0.704,0.620,0.423,1.000,185
2021-10-23,0.000,0.000,0.648,0.000,0.076,0.412,0.000,0.466,0.534,0.438,1.000,0.381,0.826,0.000,0.420,0.273,0.657,0.576,0.096,0.017,0.362,0.000,0.421,0.588,0.364,...,0.000,0.000,0.909,0.655,0.762,0.619,0.387,0.489,0.409,0.579,0.125,0.179,0.517,0.083,0.000,0.458,0.000,0.364,0.453,0.301,0.000,0.185,0.269,0.453,198
2021-10-23,0.691,0.652,0.507,0.758,0.455,0.454,0.466,0.593,0.534,0.720,0.286,0.602,0.609,0.561,0.058,0.364,0.257,0.661,0.635,0.586,0.176,0.728,0.263,0.265,0.396,...,0.676,0.758,0.398,0.276,0.720,0.591,0.032,0.744,0.370,0.660,0.375,0.357,0.638,0.708,0.238,0.246,0.581,0.682,0.433,0.749,0.728,0.076,0.423,0.023,239
2021-10-23,0.727,0.826,0.683,0.827,0.530,0.244,0.772,0.297,0.382,0.315,0.571,0.159,0.348,0.918,0.275,0.182,0.029,0.661,0.250,0.069,0.122,0.225,0.368,0.559,0.317,...,0.712,0.827,0.455,0.586,0.291,0.715,0.452,0.460,0.409,0.460,0.375,0.179,0.403,0.208,0.357,0.282,0.735,0.136,0.372,0.075,0.225,0.511,0.346,0.965,232
2021-10-24,0.745,0.783,0.577,0.848,0.833,0.580,0.820,0.254,0.229,0.560,0.357,0.779,0.826,0.765,0.565,0.818,0.543,0.322,1.000,0.897,1.000,0.362,0.842,1.000,0.559,...,0.850,0.848,0.341,0.655,0.075,1.000,0.903,0.460,0.665,1.000,0.875,0.536,0.859,0.833,0.179,0.352,0.355,1.000,1.000,0.470,0.362,0.783,0.731,0.732,204


<a name="1.-Wrapper-Methods-for-Total-Points"></a>
# 1. Wrapper Methods for Total Points

[Return to top](#Feature-Selection)

<a name="1.1.-Linear-Regression"></a>
## 1.1. Linear Regression

[Return to top](#Feature-Selection)

In [6]:
start_time = time.time()

# sequential feature selection
pts_selected_lr = utl.sequential_feature_selection(
    df=pts_features, 
    outcome_name='TOTAL_PTS', 
    estimator=LinearRegression()
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

{
    "forward_selected": [
        "ROLL_HOME_FG3_PCT",
        "ROLL_HOME_FT_PCT",
        "ROLL_HOME_PF_x",
        "ROLL_AWAY_PTS",
        "ROLL_AWAY_PF_x",
        "ROLLDIFF_HOME_FG_PCT",
        "ROLL_HOME_defensiveRating",
        "ROLL_HOME_estimatedUsagePercentage",
        "ROLL_AWAY_assistToTurnover",
        "ROLL_AWAY_PIE",
        "ROLL_HOME_CHARGES_DRAWN",
        "ROLL_HOME_OFF_LOOSE_BALLS_RECOVERED",
        "ROLL_AWAY_CONTESTED_SHOTS_2PT",
        "ROLL_AWAY_DEFLECTIONS",
        "ROLL_AWAY_DEF_BOXOUTS",
        "ROLL_HOME_PTS_FB",
        "ROLL_HOME_OPP_PTS_PAINT",
        "ROLL_HOME_BLKA",
        "ROLL_AWAY_PTS_OFF_TOV",
        "ROLL_AWAY_OPP_PTS_FB",
        "ROLL_AWAY_BLKA",
        "ROLL_AWAY_PF_y",
        "ROLL_AWAY_OPP_EFG_PCT",
        "ROLL_HOME_PCT_UAST_FGM",
        "ROLL_AWAY_PCT_FGA_2PT",
        "ROLL_AWAY_PCT_FGA_3PT",
        "ROLL_AWAY_PCT_PTS_3PT",
        "ROLL_AWAY_PCT_AST_3PM",
        "ROLL_HOME_TCHS",
        "ROLL_HOME_SAST",
        "ROLL_

<a name="1.2.-Random-Forest"></a>
## 1.2. Random Forest

[Return to top](#Feature-Selection)

In [ ]:
start_time = time.time()

# sequential feature selection
pts_selected_rf = utl.sequential_feature_selection(
    df=pts_features, 
    outcome_name='TOTAL_PTS', 
    estimator=RandomForestRegressor(random_state=599, n_jobs=-1, max_depth=8, max_features=0.3)
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

<a name="2.-Wrapper-Methods-for-Plus-Minus"></a>
# 2. Wrapper Methods for Plus Minus

[Return to top](#Feature-Selection)

<a name="2.1.-Linear-Regression"></a>
## 2.1. Linear Regression

[Return to top](#Feature-Selection)

In [ ]:
start_time = time.time()

# sequential feature selection
pm_selected_lr = utl.sequential_feature_selection(
    df=pm_features, 
    outcome_name='PLUS_MINUS', 
    estimator=LinearRegression()
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

<a name="2.2.-Random-Forest"></a>
## 2.2. Random Forest

[Return to top](#Feature-Selection)

In [ ]:
start_time = time.time()

# sequential feature selection
pm_selected_rf = utl.sequential_feature_selection(
    df=pm_features, 
    outcome_name='PLUS_MINUS', 
    estimator=RandomForestRegressor(random_state=599, n_jobs=-1, max_depth=8, max_features=0.3)
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

<a name="3.-Wrapper-Methods-for-Game-Winner"></a>
# 3. Wrapper Methods for Game Winner

[Return to top](#Feature-Selection)

<a name="3.1.-Linear-Regression"></a>
## 3.1. Linear Regression

[Return to top](#Feature-Selection)

In [ ]:
start_time = time.time()

# sequential feature selection
res_selected_lr = utl.sequential_feature_selection(
    df=res_features, 
    outcome_name='GAME_RESULT', 
    estimator=LinearRegression()
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")

<a name="3.2.-Random-Forest"></a>
## 3.2. Random Forest

[Return to top](#Feature-Selection)

In [ ]:
start_time = time.time()

# sequential feature selection
res_selected_rf = utl.sequential_feature_selection(
    df=res_features, 
    outcome_name='GAME_RESULT', 
    estimator=RandomForestClassifier(random_state=599, n_jobs=-1, max_depth=8, max_features='sqrt')
)

end_time = time.time()
print(f"Total time taken: {end_time - start_time:.2f} seconds")